# Setup & Utils

In [ ]:
import requests

In [ ]:
import re

def extract_model_response(text: str) -> str:
    """
    Extracts the content between <think> and the final <|eot_id|> marker.

    Parameters:
        text (str): The full text containing <think> and <|eot_id|> markers.

    Returns:
        str: The extracted content between <think> and <|eot_id|>, or an empty string if not found.
    """
    match = re.search(r"<think>(.*?)<\|end_of_text\|>", text, re.DOTALL)
    return match.group(1).strip() if match else ""


**Please enter the API key that was provided to you**

In [ ]:
API_KEY = ""

# Use Case - Build Environment Security

**Use Case Description** - Automate security monitoring and guidance during the software development lifecycle (SDLC) by embedding a GitHub Action that triggers on pull requests (PRs). The system summarizes changes, evaluates them for security risks, and provides actionable recommendations to reviewers. This turns every PR into an opportunity for proactive security posture improvement — not just static scanning, but contextual reasoning.

## Security Analysis Utils

In [ ]:
def create_ci_security_prompt(pr_diff_text):
    """Create a prompt to analyze security impact of a pull request diff."""
    return f"""
You are a security expert reviewing changes introduced in a pull request.

Analyze the following code diff and identify:
1. Security-relevant changes
2. Any potential vulnerabilities introduced
3. A clear summary of affected areas
4. Recommended remediations (if needed)

## PULL REQUEST DIFF
{pr_diff_text}
"""

In [ ]:
def run_ci_security_analysis(pr_diff):
    """Send the diff to an AI model for security analysis and return the response."""
    prompt = create_ci_security_prompt(pr_diff)

    resp = requests.post(
        "https://model-8w6j1ojq.api.baseten.co/development/predict",
        headers={"Authorization": f"Api-Key {API_KEY}"},
        json={"prompt": prompt},
    )

    return extract_model_response(resp.text)

## Example Usage

In this sample PR Diff, Replaced use of `eval()` with `ast.literal_eval()` — this is a significant security improvement. `eval()` can execute arbitrary code, whereas `ast.literal_eval()` safely evaluates strings with Python literals.


In [ ]:
sample_pr_diff = """
diff --git a/app/utils.py b/app/utils.py
index e69de29..b25a3c0 100644
--- a/app/utils.py
+++ b/app/utils.py
@@ def process_input(data):
-    return eval(data)
+    import ast
+    return ast.literal_eval(data)

diff --git a/.github/workflows/deploy.yml b/.github/workflows/deploy.yml
index a1b2c3d..d4e5f6g 100644
--- a/.github/workflows/deploy.yml
+++ b/.github/workflows/deploy.yml
@@ steps:
-      run: npm run deploy
+      run: |
+        set -e
+        npm run lint
+        npm test
+        npm run deploy
"""


In [ ]:
output = run_ci_security_analysis(sample_pr_diff)

In [ ]:
print(output)

**Security-Relevant Changes:**

1. The `process_input` function has been modified to use `ast.literal_eval` instead of `eval`.
2. The GitHub Actions workflow file (`deploy.yml`) now includes additional commands before running `npm run deploy`.

**Potential Vulnerabilities Introduced:**

None identified in this diff. However, there might be some minor issues:

* In the `process_input` function, using `ast.literal_eval` is generally safer than `eval`, as it only evaluates literals (strings, numbers, etc.) and not arbitrary code. But if the input data contains complex structures like lists or dictionaries with untrusted values, it may still lead to errors. To mitigate this, consider validating user input before processing.
* The GitHub Actions workflow now runs `npm test` before deploying. While this is a good practice for ensuring tests pass before deployment, it's essential to ensure that the tests cover all critical aspects of the application, including security-related checks.

**Affe